In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('chinook.db')

customer_spending_query = """
SELECT 
    c.CustomerId,
    c.FirstName || ' ' || c.LastName AS CustomerName,
    SUM(i.Total) AS TotalSpent
FROM 
    customers c
JOIN 
    invoices i ON c.CustomerId = i.CustomerId
GROUP BY 
    c.CustomerId, CustomerName
ORDER BY 
    TotalSpent DESC
LIMIT 5;
"""

top_customers = pd.read_sql(customer_spending_query, conn)
print("Top 5 Customers by Total Spending:")
print(top_customers.to_string(index=False))
print("\n")

album_vs_tracks_query = """
WITH CustomerPurchaseTypes AS (
    SELECT 
        i.CustomerId,
        CASE 
            WHEN EXISTS (
                -- Check if any invoice has tracks not matching a complete album purchase
                SELECT 1 
                FROM invoice_items ii
                JOIN tracks t ON ii.TrackId = t.TrackId
                WHERE ii.InvoiceId = i.InvoiceId
                AND NOT EXISTS (
                    -- This subquery checks if all tracks from the album are in the invoice
                    SELECT 1 
                    FROM tracks t2
                    WHERE t2.AlbumId = t.AlbumId
                    AND t2.TrackId NOT IN (
                        SELECT ii2.TrackId 
                        FROM invoice_items ii2 
                        WHERE ii2.InvoiceId = i.InvoiceId
                    )
                )
            ) THEN 'Individual Tracks'
            ELSE 'Full Albums'
        END AS PurchaseType
    FROM 
        invoices i
    GROUP BY 
        i.CustomerId
)

SELECT 
    PurchaseType,
    COUNT(*) AS CustomerCount,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM CustomerPurchaseTypes), 2) AS Percentage
FROM 
    CustomerPurchaseTypes
GROUP BY 
    PurchaseType;
"""

purchase_preferences = pd.read_sql(album_vs_tracks_query, conn)
print("Customer Purchase Preferences:")
print(purchase_preferences.to_string(index=False))

conn.close()